In [1]:
import pandas as pd
import numpy as np
from color_conversion import convert_rgb_cols
from constants import ColorSpace
from correction import CorrectionByModel
from util import load_data

df = load_data("Data/Jonas1.json")
df["lighting_condition"] = "daylight"
df = convert_rgb_cols(df, prefix="gt__", to=ColorSpace.LAB)
df = convert_rgb_cols(df, prefix="color_r4_", to=ColorSpace.LAB)

model = CorrectionByModel(space=ColorSpace.LAB, r=4, degree=1, boundary_penalty_factor=0)
model.train(df.iloc[:1].copy())
print(model.build_design_matrix())
print(model.m0, model.m1, model.m2, model.w0, model.w1, model.w2)
# order of coeffs: intercept, w2, w1, w0, m2, m1, m0, pitch, roll

Starting optimization...
Optimization finished.
[[ 1.00000000e+00  3.32769147e+01  8.61957821e+00  9.93136250e+00
   1.96000000e+02  1.99000000e+02  2.06000000e+02 -4.51998876e-02
   7.09189292e-01]]
[33.27691474] [8.61957821] [9.9313625] [196] [199] [206]


In [ ]:
from util import load_data
from plot import plot_comparison_grid, plot_against_gt, plotHSV, plotHSVError, plot_ecdf
from correction import CorrectionByModel, CorrectionByScaling
from color_conversion import convert_rgb_cols, convert_to_rgb
from constants import ColorSpace
from cross_validation import *

df_daylight1 = load_data("Data/Jonas1.json")
df_daylight1["lighting_condition"] = "daylight"
df_daylight2 = load_data("Data/Baisu1.json")
df_daylight2["lighting_condition"] = "daylight"
df_dark1 = load_data("Data/Tai4.json")
df_dark1["lighting_condition"] = "dark"
df_dark2 = load_data("Data/Zhi3.json")
df_dark2["lighting_condition"] = "dark"

df = pd.concat([df_daylight1, df_daylight2], ignore_index=True)
df_raw = df.copy()
df = convert_rgb_cols(df, prefix="gt__", to=ColorSpace.LAB)
df = convert_rgb_cols(df, prefix="color_r4_", to=ColorSpace.LAB)
df_train = df.sample(frac=0.66, random_state=0)
df_test = df.drop(df_train.index)

corrector_model_1 = CorrectionByModel(space=ColorSpace.LAB, r=4, degree=1, boundary_penalty_factor=0, pose=True)
corrector_model_1.train(df_train.copy())
df_test = corrector_model_1.apply_correction(df_test.copy())

# check if outside of boundary


In [2]:
coeffs = np.load('coeffs.npz')['coefficients']
coeffs_with_ci = pd.read_csv('model_parameters_95CI.csv', index_col=0)
coeffs_with_ci['estimate'] = coeffs.flatten()
# coeffs = coeffs_with_ci[:, 0].reshape(-1, 3).T
cols = ['(offset)', 'b_ref', 'a_ref', 'L_ref', 'b_meas', 'a_meas', 'L_meas', 'pitch', 'roll']
rows = ['L', 'a', 'b']
# coeffs = pd.DataFrame(coeffs, index=rows, columns=cols).reindex(
#     columns=['(offset)', 'L_meas', 'a_meas', 'b_meas', 'L_ref', 'a_ref', 'b_ref', 'pitch', 'roll']
# )

# ci = coeffs_with_ci[:, 1:].reshape(-1, 3, 2).transpose(1, 0, 2)
# ci.shape
coeffs_with_ci = coeffs_with_ci.apply(
    lambda row: f"{row['estimate']:6.2f} [{row['ci_025']:6.2f}, {row['ci_975']:6.2f}]",
    axis=1
).to_numpy().reshape(-1, 3).T
coeffs_with_ci = pd.DataFrame(coeffs_with_ci, index=rows, columns=cols).reindex(
    columns=['(offset)', 'L_meas', 'a_meas', 'b_meas', 'L_ref', 'a_ref', 'b_ref', 'pitch', 'roll']
)
coeffs_with_ci

,(offset),L_meas,a_meas,b_meas,L_ref,a_ref,b_ref,pitch,roll
L,"69.46 [ 0.11, 81.88]","1.25 [ 1.21, 1.31]","-0.14 [ -0.18, -0.10]","0.11 [ 0.07, 0.13]","0.50 [ -0.35, 0.76]","-1.10 [ -1.61, 0.54]","0.25 [ -0.24, 0.51]","-0.06 [ -0.15, -0.03]","0.00 [ -0.04, 0.04]"
a,"-16.82 [-27.10, 0.01]","0.03 [ 0.00, 0.07]","1.32 [ 1.29, 1.35]","-0.21 [ -0.24, -0.18]","-0.78 [ -1.02, -0.47]","1.38 [ 0.65, 1.89]","-0.50 [ -0.75, -0.19]","0.02 [ -0.03, 0.08]","0.00 [ -0.03, 0.04]"
b,"5.45 [ -7.28, 20.33]","-0.10 [ -0.14, -0.04]","-0.08 [ -0.14, -0.03]","1.40 [ 1.38, 1.44]","0.26 [ -0.11, 0.58]","-0.81 [ -1.44, -0.18]","0.52 [ 0.22, 0.83]","0.07 [ 0.01, 0.14]","-0.02 [ -0.06, 0.02]"


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from util import load_data
from plot import plot_comparison_grid, plot_against_gt, plotHSV, plotHSVError, plot_ecdf
from correction import CorrectionByModel, CorrectionByScaling
from color_conversion import convert_rgb_cols, convert_to_rgb
from constants import ColorSpace
from cross_validation import *

df_daylight1 = load_data("Data/Jonas1.json")
df_daylight1["lighting_condition"] = "daylight"
df_daylight2 = load_data("Data/Baisu1.json")
df_daylight2["lighting_condition"] = "daylight"
df_dark1 = load_data("Data/Tai4.json")
df_dark1["lighting_condition"] = "dark"
df_dark2 = load_data("Data/Zhi3.json")
df_dark2["lighting_condition"] = "dark"

#-------------------------------------------------------------------------------
# df = pd.concat([df_daylight1, df_daylight2, df_dark1, df_dark2], ignore_index=True)
# corrector0, corrector2, corrector4 = CorrectionByScaling(space=ColorSpace.RGB, r=0), CorrectionByScaling(space=ColorSpace.RGB, r=2), CorrectionByScaling(space=ColorSpace.RGB, r=4)
# df_corrected = corrector4.predict(corrector2.predict(corrector0.predict(df)))
# fig, ax = plot.plot_comparison_grid(df_corrected, radius=4, rows=4, cols=6)
# #fig, ax = plot.plotHSV(df_corrected)
# #fig, ax = plot.plotHSVError(df_corrected)
#-------------------------------------------------------------------------------

df = pd.concat([df_daylight1, df_daylight2], ignore_index=True)
df_raw = df.copy()
df = convert_rgb_cols(df, prefix="gt__", to=ColorSpace.LAB)
df = convert_rgb_cols(df, prefix="color_r4_", to=ColorSpace.LAB)
df = convert_rgb_cols(df, prefix="white_r4_", to=ColorSpace.LAB)
df_train = df.sample(frac=0.66, random_state=0)
df_test = df.drop(df_train.index)

In [ ]:
from statsmodels.multivariate.multivariate_ols import _MultivariateOLS

design = df_train[[
    "color_r4_l", "color_r4_a", "color_r4_b",
    "white_r4_l", "white_r4_a", "white_r4_b",
    # "gt__l", "gt__a", "gt__b",
    "pitch", "roll"
]]
target = df_train[[
    "gt__l", "gt__a", "gt__b"
]]

model = _MultivariateOLS(target, design)
results = model.fit()
params, df_resid, inv_cov, sscpr = results._fittedmod

array([[ 9791.01194109,  -108.55132032,  1093.12246661],
       [ -108.55132032,  7383.84177461, -3938.72592789],
       [ 1093.12246661, -3938.72592789, 12976.3075095 ]])